# Analysis of ambiental radioactivity ntuples by Matthew Green
In this notebook, we analyse the events contained in the root files by Matt Green about ambiental radioactivity.

TODO list:
1. adapt python script to ntuple output scheme
1. run the script to extract csv from root
1. start analysis